In [5]:
import osmnx as ox
import networkx as nx
import numpy as np
import folium
import heapq
import random
import os
import time
import copy
import math
import csv

In [6]:
def generate_map_matrix(num_vertices, location, data_folder):
    """Generate adjacency matrix of a street network graph from OpenStreetMap data."""

    # Download the street network data from OSM
    graph = ox.graph_from_place(location, network_type='drive')

    # Ensure we have at least 100 nodes and 100 edges
    nodes = list(graph.nodes)
    edges = list(graph.edges)
    if len(nodes) < num_vertices or len(edges) < num_vertices:
        raise Exception("The graph does not have the required number of nodes or edges.")

    # Convert graph to adjacency matrix
    adj_matrix = nx.to_numpy_array(graph, nodelist=nodes)
    
    # Save the matrix to a file
    np.save(os.path.join(data_folder, 'map_matrix.npy'), adj_matrix)

    # Save node positions
    node_positions = {i: (graph.nodes[node]['y'], graph.nodes[node]['x']) for i, node in enumerate(nodes)}
    np.save(os.path.join(data_folder, 'node_positions.npy'), node_positions)

    return adj_matrix, node_positions, nodes


def count_edges(adj_matrix):
    # Count number of non-zero elements in the adjacency matrix
    num_edges = np.count_nonzero(adj_matrix)
    
    # If the graph is undirected, divide by 2
    if np.array_equal(adj_matrix, adj_matrix.T):
        num_edges //= 2

    return num_edges

def draw_path(node_positions, path, start_coord, end_coord, data_folder, name):
    """Draw a path on the map."""

    # Create a folium map centered around the start point
    folium_map = folium.Map(location=[start_coord[0], start_coord[1]], zoom_start=14)

    # Add start and end points to the map
    folium.CircleMarker(location=[start_coord[0], start_coord[1]], radius=5, popup='Start Point', color='black', fill=True, fill_color='white').add_to(folium_map)
    folium.Marker(location=[end_coord[0], end_coord[1]], popup='End Point', icon=folium.Icon(color='red')).add_to(folium_map)

    # Draw the path on the map
    latlons = [node_positions[node] for node in path]
    folium.PolyLine(locations=latlons, color='blue', popup=name).add_to(folium_map)

    # Save the map with the path
    folium_map.save(os.path.join(data_folder, f'{name}.html'))

    print(f"The map with {name} algorithm has been saved to {data_folder}{name}.html.")

def generate_random_pairs(n, k):
    vertices = list(range(n))
    pairs = [random.sample(vertices, 2) for _ in range(k)]
    return pairs

def dijkstra(adj_matrix, source, target):
    n = len(adj_matrix)
    dist = [float('inf')] * n
    prev = [None] * n
    dist[source] = 0
    queue = [(0, source)]

    while queue:
        current_dist, current_vertex = heapq.heappop(queue)

        if current_dist > dist[current_vertex]:
            continue

        for neighbor, weight in enumerate(adj_matrix[current_vertex]):
            if weight == 0.0:  # Skip if there is no edge
                continue

            alt_dist = current_dist + weight

            if alt_dist < dist[neighbor]:
                dist[neighbor] = alt_dist
                prev[neighbor] = current_vertex
                heapq.heappush(queue, (alt_dist, neighbor))

    path = []
    while target is not None:
        path.append(target)
        target = prev[target]
    path.reverse()

    return path

def convert_adj_matrix_to_edges(adj_matrix):
    num_vertices = len(adj_matrix)
    edges = []
    for u in range(num_vertices):
        for v in range(num_vertices):
            if adj_matrix[u][v] != 0.0:  # There is an edge
                edges.append((u, v, adj_matrix[u][v]))
    return edges

def bellman_ford(edges, num_vertices, source, target):
    dist = [float('inf')] * num_vertices
    prev = [None] * num_vertices
    dist[source] = 0

    # Relax all edges num_vertices times
    for i in range(num_vertices):
        for u, v, weight in edges:
            if dist[u] != float('inf') and dist[u] + weight < dist[v]:
                if i == num_vertices - 1:
                    raise ValueError("Graph contains a negative-weight cycle")
                dist[v] = dist[u] + weight
                prev[v] = u

    # Reconstruct the shortest path
    path = []
    current_vertex = target
    while current_vertex is not None:
        path.append(current_vertex)
        current_vertex = prev[current_vertex]
    path.reverse()

    return path

def floyd_warshall_preprocess(adj_matrix):
    n = len(adj_matrix)
    dist = [[float('inf')] * n for _ in range(n)]
    next_vertex = [[None] * n for _ in range(n)]

    for u in range(n):
        for v, weight in enumerate(adj_matrix[u]):
            if weight != 0:  # There is an edge
                dist[u][v] = weight
                next_vertex[u][v] = v

    for k in range(n):
        for i in range(n):
            for j in range(n):
                if dist[i][k] + dist[k][j] < dist[i][j]:
                    dist[i][j] = dist[i][k] + dist[k][j]
                    next_vertex[i][j] = next_vertex[i][k]

    return next_vertex, dist

def floyd_warshall_path(next_vertex, source, target):
    path = []
    while source != target:
        if next_vertex[source][target] is None:
            return None  # No path
        path.append(source)
        source = next_vertex[source][target]
    path.append(target)

    return path

### Initialize adjacency matrix for map data

In [7]:
location = 'District 11, Ho Chi Minh City, Vietnam'
num_vertices = 100
data_folder = './data/'
result_folder = './results/'
os.makedirs(data_folder, exist_ok=True)
os.makedirs(result_folder, exist_ok=True)

# Generate adjacency matrix and node positions
adj_matrix, node_positions, nodes = generate_map_matrix(num_vertices, location, data_folder)

print('Number of nodes:', len(node_positions))
print('Number of edges:', count_edges(adj_matrix))

Number of nodes: 1139
Number of edges: 2764


In [100]:
def check_matrix_values(adj_matrix):

    def print_other_values_and_positions(adj_matrix):
        other_values_positions = np.where((adj_matrix != 0.0) & (adj_matrix != 1.0))
        other_values = adj_matrix[other_values_positions]
        
        for position, value in zip(zip(*other_values_positions), other_values):
            print(f"Value: {value} at position {position}")

    if np.all((adj_matrix == 0.0) | (adj_matrix == 1.0)):
        print("The matrix contains only 0.0 and 1.0")
    elif np.any(np.isnan(adj_matrix)):
        print("The matrix contains null values")
    else:
        print("The matrix contains other values")
        print_other_values_and_positions(adj_matrix)

# Test the function
check_matrix_values(adj_matrix)

The matrix contains other values
Value: 2.0 at position (216, 1121)
Value: 2.0 at position (306, 973)
Value: 2.0 at position (380, 903)
Value: 2.0 at position (559, 560)
Value: 2.0 at position (560, 559)
Value: 2.0 at position (864, 864)
Value: 2.0 at position (903, 380)
Value: 2.0 at position (947, 948)
Value: 2.0 at position (948, 947)
Value: 2.0 at position (973, 306)
Value: 2.0 at position (1028, 1029)
Value: 2.0 at position (1029, 1028)
Value: 2.0 at position (1121, 216)


### Compare algorithms performance

In [197]:
n = len(adj_matrix)
num_pairs = 5

node_pairs = generate_random_pairs(n, num_pairs)

# Initialize performance results
performance_results = {
    'Dijkstra': [],
    'Bellman-Ford': [],
    'Floyd-Warshall': []
}

# Measure performance for Dijkstra's algorithm
i = 0
for source, target in node_pairs:
    i += 1
    start_time = time.time()
    path, _ = dijkstra(adj_matrix, source, target)
    duration = time.time() - start_time
    if len(path) <= 1:
        performance_results['Dijkstra'].append(float('inf'))
    else:
        performance_results['Dijkstra'].append(duration)

        # Get the latitude and longitude of the source and target nodes
        source_lat, source_lon = node_positions[source][0], node_positions[source][1]
        target_lat, target_lon = node_positions[target][0], node_positions[target][1]

        # print(source_lat, source_lon, target_lat, target_lon)
        # print(path)

        draw_path(node_positions, path, (source_lat, source_lon), (target_lat, target_lon), result_folder, f'dijkstra_{i}')


# Measure performance for Bellman-Ford's algorithm
list_edges = convert_adj_matrix_to_edges(adj_matrix)
num_vertices = len(adj_matrix)
i = 0
for source, target in node_pairs:
    i += 1
    start_time = time.time()
    path = bellman_ford(list_edges, num_vertices, source, target)
    duration = time.time() - start_time
    if len(path) <= 1:
        performance_results['Bellman-Ford'].append(float('inf'))
    else:
        performance_results['Bellman-Ford'].append(duration)

        # Get the latitude and longitude of the source and target nodes
        source_lat, source_lon = node_positions[source][0], node_positions[source][1]
        target_lat, target_lon = node_positions[target][0], node_positions[target][1]

        # print(source_lat, source_lon, target_lat, target_lon)
        # print(path)

        draw_path(node_positions, path, (source_lat, source_lon), (target_lat, target_lon), result_folder, f'bellman_ford_{i}')


# Measure performance for Bellman-Ford's algorithm
start_time = time.time()
next_vertex, dist = floyd_warshall_preprocess(adj_matrix)
preprocess_time = time.time() - start_time

i = 0
for source, target in node_pairs:
    i += 1
    if source < len(next_vertex) and target < len(next_vertex[source]):
        start_time = time.time()
        path = floyd_warshall_path(next_vertex, source, target)
        floyd_warshall_duration = time.time() - start_time + preprocess_time

        if len(path) > 1:
            performance_results['Floyd-Warshall'].append(floyd_warshall_duration)
        else:
            performance_results['Floyd-Warshall'].append(float('inf'))
            continue

        # Get the latitude and longitude of the source and target nodes
        source_lat, source_lon = node_positions[source][0], node_positions[source][1]
        target_lat, target_lon = node_positions[target][0], node_positions[target][1]

        # print(source_lat, source_lon, target_lat, target_lon)
        # print(path)

        draw_path(node_positions, path, (source_lat, source_lon), (target_lat, target_lon), result_folder, f'floyd_warshall_{i}')
    else:
        performance_results['Floyd-Warshall'].append(float('inf'))

# Print performance results
for algo, durations in performance_results.items():
    print(f"{algo} algorithm performance:")
    for i, duration in enumerate(durations):
        print(f"  Pair {i+1}: {duration:.6f} seconds")

# Save performance results to a file
file_path = os.path.join(result_folder, 'performance_results.csv')
with open(file_path, 'w', newline='') as csvfile:
    fieldnames = ['Algorithm', 'Pair', 'Duration']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for algo, durations in performance_results.items():
        for i, duration in enumerate(durations):
            writer.writerow({'Algorithm': algo, 'Pair': i+1, 'Duration': duration})

print(f"Performance results have been saved to {file_path}.")

The map with dijkstra_1 algorithm has been saved to ./results/dijkstra_1.html.
The map with dijkstra_2 algorithm has been saved to ./results/dijkstra_2.html.
The map with dijkstra_3 algorithm has been saved to ./results/dijkstra_3.html.
The map with dijkstra_4 algorithm has been saved to ./results/dijkstra_4.html.
The map with dijkstra_5 algorithm has been saved to ./results/dijkstra_5.html.
The map with bellman_ford_1 algorithm has been saved to ./results/bellman_ford_1.html.
The map with bellman_ford_2 algorithm has been saved to ./results/bellman_ford_2.html.
The map with bellman_ford_3 algorithm has been saved to ./results/bellman_ford_3.html.
The map with bellman_ford_4 algorithm has been saved to ./results/bellman_ford_4.html.
The map with bellman_ford_5 algorithm has been saved to ./results/bellman_ford_5.html.
The map with floyd_warshall_1 algorithm has been saved to ./results/floyd_warshall_1.html.
The map with floyd_warshall_2 algorithm has been saved to ./results/floyd_warsh

### Generate 3 shortest paths between 2 arbitrary coordinates

In [42]:
def generate_interactive_map(location, data_folder):
    """Generate an interactive map of a street network graph from OpenStreetMap data to select start and end points."""

    # Download the street network data from OSM
    graph = ox.graph_from_place(location, network_type='drive')

    # Convert the graph to an undirected graph for simplicity
    graph = graph.to_undirected()

    # Create a folium map centered around District 10
    center_point = ox.geocode(location)
    folium_map = folium.Map(location=center_point, zoom_start=14)

    # Add a clickable feature to the map
    folium_map.add_child(folium.ClickForMarker())

    # Add the edges to the map
    for u, v, data in graph.edges(data=True):
        # Get the coordinates of the nodes
        u_coords = (graph.nodes[u]['y'], graph.nodes[u]['x'])
        v_coords = (graph.nodes[v]['y'], graph.nodes[v]['x'])
        
        # Draw the edge on the map
        folium.PolyLine(locations=[u_coords, v_coords], color='blue').add_to(folium_map)

    # Save the initial map
    folium_map.save(os.path.join(data_folder, 'initial_map.html'))

    print(f"Please open {data_folder}initial_map.html and click on the map to get coordinates for start and end points.")

def nearest_node(node_positions, coord):
    """Find the node nearest to the provided coordinate."""
    min_distance = float('inf')
    nearest = None
    for node, position in node_positions.items():
        distance = math.sqrt((position[0] - coord[0])**2 + (position[1] - coord[1])**2)
        if distance < min_distance:
            min_distance = distance
            nearest = node
    return nearest

def yen_ksp(adj_matrix, source, target, K):
    A = []
    B = []

    # Step 1: Find the shortest path from source to target
    shortest_path = dijkstra(adj_matrix, source, target)
    if not shortest_path:
        return A

    A.append(shortest_path)
    for k in range(1, K):
        for i in range(len(A[-1]) - 1):
            spur_node = A[-1][i]
            root_path = A[-1][:i + 1]

            # Remove the edges that are part of the previous shortest paths
            removed_edges = []
            for path in A:
                if path[:i + 1] == root_path and len(path) > (i + 1):
                    weight = adj_matrix[path[i]][path[i + 1]]
                    adj_matrix[path[i]][path[i + 1]] = 0
                    removed_edges.append((path[i], path[i + 1], weight))

            spur_path = dijkstra(adj_matrix, spur_node, target)

            # Add the potential k-th shortest path 
            # if it is not a duplicate of a previously found path
            if spur_path:
                total_path = root_path[:-1] + spur_path
                if total_path not in B:
                    heapq.heappush(B, (len(total_path), total_path))

            # Restore the adjacency matrix
            for edge in removed_edges:
                adj_matrix[edge[0]][edge[1]] = edge[2]

        if B:
            A.append(heapq.heappop(B)[1])
        else:
            break

    return A

def draw_shortest_paths(node_positions, paths, start_coord, end_coord, data_folder):
    """Draw the k shortest paths on the map."""

    # Create a folium map centered around the start point
    folium_map = folium.Map(location=start_coord, zoom_start=14)

    # Draw the 3 shortest paths on the map
    paths = paths[::-1]
    colors = ['red', 'lightgreen', 'blue']

    for i, path in enumerate(paths):
        latlons = [node_positions[node] for node in path]
        folium.PolyLine(locations=latlons, color=colors[i % len(colors)], popup=f"Path {i+1}", weight=5).add_to(folium_map)

    # Add start and end points to the map
    # folium.CircleMarker(location=start_coord, radius=8, popup='Start Point', color='black', fill=True, fill_color='white').add_to(folium_map)
    folium.Marker(location=start_coord, popup=f'{start_coord[0]:.5f}, {start_coord[1]:.5f}', icon=folium.Icon(color='green')).add_to(folium_map)
    folium.Marker(location=end_coord, popup=f'{end_coord[0]:.5f}, {end_coord[1]:.5f}', icon=folium.Icon(color='red')).add_to(folium_map)

    # Save the map with paths
    folium_map.save(os.path.join(data_folder, 'shortest_paths.html'))

    print(f"The map with {i + 1} shortest paths has been saved to {data_folder}shortest_paths.html.")


def find_k_shortest_paths(adj_matrix, node_positions, start_coord, end_coord, data_folder, k):
    """Find the 3 shortest paths between the start and end points."""

    # Find nearest nodes to the provided coordinates
    start_node = nearest_node(node_positions, start_coord)
    end_node = nearest_node(node_positions, end_coord)

    # Find the 3 shortest paths between the start and end nodes
    start = time.time()
    shortest_paths = yen_ksp(adj_matrix, start_node, end_node, k)
    duration = time.time() - start

    # # Draw the 3 shortest paths on the map
    draw_shortest_paths(node_positions, shortest_paths, start_coord, end_coord, data_folder)

    return shortest_paths, duration

In [209]:
# Generate an interactive map to select start and end points
generate_interactive_map(location, result_folder)

Please open ./results/initial_map.html and click on the map to get coordinates for start and end points.


In [44]:
start_coord = 10.76391969853238, 106.65567874574552
end_coord =  10.763017717931572, 106.64984353210734

# Find the 3 shortest paths between the start and end points
shortest_paths, duration = find_k_shortest_paths(adj_matrix, node_positions, list(start_coord), list(end_coord), result_folder, 3)
print(f'Time for generating 3 shortest paths: {duration:.2f} seconds.')

The map with 3 shortest paths has been saved to ./results/shortest_paths.html.
Time for generating 3 shortest paths: 3.86 seconds.
